### Importing Libraries

In [14]:
import numpy as np
from ultralytics import YOLO
import cv2 as cv
import cvzone
import math
from sort import *

**Using a video as reference to check instead of a live camera.**

In [25]:
cap = cv.VideoCapture("people.mp4")

### Creating an object of the YOLO model

In [16]:
model = YOLO("../Yolo-Weights/yolov8l.pt")

In [17]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

**Created a mask with the help of Canva.**

In [18]:
mask = cv.imread("mask.png")

**Tracking the individuals according to their IDs, to make sure each unique person is counted only once.**

In [19]:
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

In [20]:
limitsUp = [103, 161, 296, 161]
limitsDown = [527, 489, 735, 489]

In [26]:
totalCountUp = []
totalCountDown = []
 
while True:
    success, img = cap.read()
    imgRegion = cv.bitwise_and(img, mask)
 
    imgGraphics = cv.imread("graphics.png", cv.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (730, 260))
    results = model(imgRegion, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box around the person
            x1, y1, x2, y2 = box.xyxy[0] 
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
 
            if currentClass == "person" and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray)) # appending the detected values into the numpy array
 
    resultsTracker = tracker.update(detections)

    # creating the reference lines, which surpassed, then the count increases by one
    cv.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 0, 255), 3)
    cv.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 0, 255), 3)
 
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(100, 150, 0))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=2, offset=10)
 
        cx, cy = x1 + w // 2, y1 + h // 2
        cv.circle(img, (cx, cy), 2, (255, 255, 255), cv.FILLED) # creating a reference dot at the centre of the rectangle potraying the person
 
        if limitsUp[0] < cx < limitsUp[2] and limitsUp[1] - 15 < cy < limitsUp[1] + 15: # to check if the centre circle passes through the line
            if totalCountUp.count(id) == 0: # checks if that particular ID is present or no
                totalCountUp.append(id)
                cv.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 255, 0), 3) # changes the color of the line when person passes
 
        if limitsDown[0] < cx < limitsDown[2] and limitsDown[1] - 15 < cy < limitsDown[1] + 15:
            if totalCountDown.count(id) == 0:
                totalCountDown.append(id)
                cv.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 255, 0), 5)
    
    cv.putText(img,str(len(totalCountUp)),(929,345),cv.FONT_HERSHEY_PLAIN,5,(139,195,75),7)
    cv.putText(img,str(len(totalCountDown)),(1191,345),cv.FONT_HERSHEY_PLAIN,5,(50,50,230),7)
 
    cv.imshow("Image", img)
    
    cv.waitKey(1)


0: 384x640 2 persons, 2942.0ms
Speed: 15.1ms preprocess, 2942.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1116.4ms
Speed: 0.0ms preprocess, 1116.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[     395.22      645.12      529.45      716.46          24]

0: 384x640 2 persons, 1070.9ms
Speed: 0.0ms preprocess, 1070.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[     394.01       641.5      531.85       717.3          24]

0: 384x640 2 persons, 1069.2ms
Speed: 0.0ms preprocess, 1069.2ms inference, 9.1ms postprocess per image at shape (1, 3, 384, 640)
[     230.33      313.78      350.81       575.6          25]
[     394.57      638.02      534.28      717.57          24]

0: 384x640 2 persons, 1070.1ms
Speed: 0.0ms preprocess, 1070.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[     228.21      310.83      349.11      573.28          25]
[     393.82      635.02      534.

KeyboardInterrupt: 